<a href="https://colab.research.google.com/github/newgabo/DM_CyT_TP_1/blob/main/TP1_DMCyT_Gabriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Para montar el Google Drive y acceder a los archivos.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Instalaciones
!pip -q install spotipy
!pip -q install -U librosa seaborn
#!wget -q https://www.dropbox.com/s/nqjs076fidighaz/SopSax.NoVib.mf.C4B4.wav?dl=1 -O SopSax.NoVib.mf.C4B4.wav
!pip install feature_engine

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 
     |████████████████████████████████| 9.5MB 5.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
# Librerias
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials
from numpy import asarray
from feature_engine import variable_transformers as vt
import statsmodels.api as sm

# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d
from scipy import stats 

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import QuantileTransformer

In [3]:
# Bajamos los archivos del Dropbox
!wget https://www.dropbox.com/s/ms6260jqmsmz5qu/audio_features.pickle?dl=1 -O audio_features.pickle
!wget https://www.dropbox.com/s/8d0y9bms80ly8ui/tracks.pickle?dl=1 -O tracks.pickle
!wget https://www.dropbox.com/s/1cmb8e8zvv6hsfs/audio_analysis.pickle?dl=1 -O audio_analysis.pickle

--2020-10-29 22:12:14--  https://www.dropbox.com/s/ms6260jqmsmz5qu/audio_features.pickle?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/ms6260jqmsmz5qu/audio_features.pickle [following]
--2020-10-29 22:12:14--  https://www.dropbox.com/s/dl/ms6260jqmsmz5qu/audio_features.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc48835c430986d7a04800238287.dl.dropboxusercontent.com/cd/0/get/BCNNKYRYdNT_pO3TvGYs0Ddork25RFTE9p6c4NXaLonOeAkdLOzjKWrD9KQcsBz-vDF01kGW6QYrW4qW_5Pp53evedrp9dAlQ4ea9KUcLUpS2VDDG5Oy9Zj8CmiCwVIBe_8/file?dl=1# [following]
--2020-10-29 22:12:15--  https://uc48835c430986d7a04800238287.dl.dropboxusercontent.com/cd/0/get/BCNNKYRYdNT_pO3TvGYs0Ddork25RFTE9p6c4NXaLonOeAkdLOzjKWrD9KQcsBz-vDF01kGW6QYrW4qW_5

In [4]:
df_tracks = pd.read_pickle('tracks.pickle')
df_af = pd.read_pickle('audio_features.pickle')
df_aa = pd.read_pickle('audio_analysis.pickle')

## PROCESAMIENTO DEL DATASET AUDIO ANALISIS

In [5]:
df_aa.head()

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,pitches,timbre,id
0,0.00000,0.33061,0.000,-15.742,0.01269,-12.896,0.0,"[0.102, 0.103, 0.112, 0.21, 1.0, 0.068, 0.045,...","[43.022, 16.787, -40.89, 11.82, 105.068, -67.1...",6avAsaa2IsXHT2mtQ6cGVe
1,0.33061,0.36481,0.159,-22.482,0.08965,-18.792,0.0,"[0.008, 0.022, 0.029, 0.191, 1.0, 0.078, 0.013...","[38.644, -65.426, -9.06, -7.153, 135.949, -47....",6avAsaa2IsXHT2mtQ6cGVe
2,0.69542,0.53002,1.000,-23.472,0.00965,-7.144,0.0,"[0.037, 0.047, 0.063, 0.307, 1.0, 0.149, 0.06,...","[47.643, 50.817, 75.922, 53.108, 25.988, -42.0...",6avAsaa2IsXHT2mtQ6cGVe
3,1.22544,0.44059,0.626,-19.710,0.00809,-10.394,0.0,"[0.045, 0.067, 0.048, 0.131, 0.464, 0.104, 0.0...","[44.839, -20.55, 58.387, 38.347, 47.279, -59.7...",6avAsaa2IsXHT2mtQ6cGVe
4,1.66603,0.26816,0.082,-21.887,0.11164,-18.096,0.0,"[0.022, 0.045, 0.036, 0.149, 0.695, 0.147, 0.0...","[40.058, -29.617, 61.054, -12.254, 110.751, -4...",6avAsaa2IsXHT2mtQ6cGVe


In [21]:

# Nos quedamos solo con las columnas que nos interesan
df_aa_summary = df_aa[['pitches', 'timbre', 'id', 'confidence']].head(10000)
 
# Iteramos para separa cada valor y obtener una columna por cada uno de los pitches
for i in range(0,12):
  df_aa_summary['pitch_' + str(i)] = df_aa_summary.apply(lambda x : x['pitches'][i], axis=1) 

# Iteramos para separa cada valor y obtener una columna por cada uno de los timbre
for i in range(0,12):
  df_aa_summary['timbre_' + str(i)] = df_aa_summary.apply(lambda x : x['timbre'][i], axis=1)

  # Creamos un nuevo dataframe para calcular sobre este la media agrupado por ID de tema
df_aa_summary = df_aa_summary[['id', 'confidence',
                   'pitch_0', 'pitch_1', 'pitch_2', 'pitch_3', 'pitch_4', 'pitch_5', 
                   'pitch_6', 'pitch_7', 'pitch_8', 'pitch_9', 'pitch_10', 'pitch_11', 
                   'timbre_0', 'timbre_1', 'timbre_2', 'timbre_3', 'timbre_4', 'timbre_5', 
                   'timbre_6', 'timbre_7', 'timbre_8', 'timbre_9', 'timbre_10', 'timbre_11']]


In [22]:
df_aa_summary.head()

,id,confidence,pitch_0,pitch_1,pitch_2,pitch_3,pitch_4,pitch_5,pitch_6,pitch_7,pitch_8,pitch_9,pitch_10,pitch_11,timbre_0,timbre_1,timbre_2,timbre_3,timbre_4,timbre_5,timbre_6,timbre_7,timbre_8,timbre_9,timbre_10,timbre_11
0,6avAsaa2IsXHT2mtQ6cGVe,0.000,0.102,0.103,0.112,0.210,1.000,0.068,0.045,0.101,0.197,0.764,0.148,0.063,43.022,16.787,-40.890,11.820,105.068,-67.123,-46.124,-19.581,-28.825,-10.017,-9.831,7.593
1,6avAsaa2IsXHT2mtQ6cGVe,0.159,0.008,0.022,0.029,0.191,1.000,0.078,0.013,0.016,0.144,0.776,0.042,0.020,38.644,-65.426,-9.060,-7.153,135.949,-47.341,4.971,2.900,-22.942,-7.643,-10.601,16.728
2,6avAsaa2IsXHT2mtQ6cGVe,1.000,0.037,0.047,0.063,0.307,1.000,0.149,0.060,0.106,0.214,0.731,0.127,0.055,47.643,50.817,75.922,53.108,25.988,-42.029,-32.740,-32.456,-48.101,-6.274,-23.349,-1.042
3,6avAsaa2IsXHT2mtQ6cGVe,0.626,0.045,0.067,0.048,0.131,0.464,0.104,0.053,0.058,0.137,1.000,0.084,0.099,44.839,-20.550,58.387,38.347,47.279,-59.790,-35.080,-17.340,-17.467,-4.857,-17.286,17.836
4,6avAsaa2IsXHT2mtQ6cGVe,0.082,0.022,0.045,0.036,0.149,0.695,0.147,0.039,0.051,0.188,1.000,0.070,0.040,40.058,-29.617,61.054,-12.254,110.751,-42.497,-12.404,8.854,0.289,3.434,-5.142,16.783


In [15]:
wtavg = lambda x: np.average(x[['pitch_0', 'pitch_1', 'pitch_2', 'pitch_3', 'pitch_4', 'pitch_5', 
                   'pitch_6', 'pitch_7', 'pitch_8', 'pitch_9', 'pitch_10', 'pitch_11', 
                   'timbre_0', 'timbre_1', 'timbre_2', 'timbre_3', 'timbre_4', 'timbre_5', 
                   'timbre_6', 'timbre_7', 'timbre_8', 'timbre_9', 'timbre_10', 'timbre_11']], 
                   weights = x.confidence, axis = 0)


In [63]:
aa_weight_mean_serie = df_aa_summary.groupby('id').apply(wtavg)
aa_weight_mean_df = pd.DataFrame(aa_weight_mean_serie)
for i in range(0,24):
  aa_weight_mean_df['col_' + str(i)] = aa_weight_mean_df.apply(lambda x : x[0][i], axis=1) 
aa_weight_mean_df = aa_weight_mean_df.drop(columns=[0])

In [68]:
aa_weight_mean_df.columns = ['pitch_0_Wmean', 'pitch_1_Wmean', 'pitch_2_Wmean', 'pitch_3_Wmean', 'pitch_4_Wmean', 'pitch_5_Wmean', 
                   'pitch_6_Wmean', 'pitch_7_Wmean', 'pitch_8_Wmean', 'pitch_9_Wmean', 'pitch_10_Wmean', 'pitch_11_Wmean', 
                   'timbre_0_Wmean', 'timbre_1_Wmean', 'timbre_2_Wmean', 'timbre_3_Wmean', 'timbre_4_Wmean', 'timbre_5_Wmean', 
                   'timbre_6_Wmean', 'timbre_7_Wmean', 'timbre_8_Wmean', 'timbre_9_Wmean', 'timbre_10_Wmean', 'timbre_11_Wmean']

In [69]:
aa_weight_mean_df.head()

,pitch_0_Wmean,pitch_1_Wmean,pitch_2_Wmean,pitch_3_Wmean,pitch_4_Wmean,pitch_5_Wmean,pitch_6_Wmean,pitch_7_Wmean,pitch_8_Wmean,pitch_9_Wmean,pitch_10_Wmean,pitch_11_Wmean,timbre_0_Wmean,timbre_1_Wmean,timbre_2_Wmean,timbre_3_Wmean,timbre_4_Wmean,timbre_5_Wmean,timbre_6_Wmean,timbre_7_Wmean,timbre_8_Wmean,timbre_9_Wmean,timbre_10_Wmean,timbre_11_Wmean
id,,,,,,,,,,,,,,,,,,,,,,,,
1DMIHggF47tqsO468VUDmd,0.435220,0.261965,0.262237,0.213224,0.395815,0.188345,0.343309,0.400102,0.228654,0.253433,0.234459,0.405078,45.147693,21.731224,24.083257,-31.932006,25.826342,-18.781145,-13.161941,1.243882,7.787331,1.681014,-5.828593,0.960786
33JZQNVxMINcDuu2koRXtr,0.370442,0.285641,0.260319,0.341380,0.285412,0.405948,0.404191,0.520170,0.391876,0.324756,0.327100,0.250162,50.993739,88.004010,14.220965,6.929735,16.905188,-33.276949,9.113656,6.307409,-1.152088,8.644602,-9.942668,-1.715168
3UPgBavVpGvlPclHJNrqsb,0.336109,0.281142,0.260963,0.199536,0.212485,0.481632,0.276989,0.188262,0.131727,0.231434,0.390027,0.221069,36.372939,19.768234,78.302440,-18.922133,64.515674,-8.180532,-19.743836,-1.261417,10.632512,3.441897,-6.991157,-9.364693
3bT5PDBhVj4ifU11zQvGP2,0.442901,0.267598,0.373381,0.272158,0.247102,0.397477,0.265433,0.407457,0.259464,0.316433,0.504538,0.421908,51.123855,65.908883,10.598701,-6.662074,6.490245,-27.133563,-9.145951,-0.197619,-13.297362,3.624524,-7.164732,-11.804236
5GmrXcVIMgLaaqKVxhi5kp,0.276638,0.352071,0.453711,0.197616,0.293614,0.221086,0.438157,0.364622,0.227181,0.341916,0.197917,0.384561,37.362789,54.351273,-27.819448,-28.508598,13.360705,-16.915146,-11.772864,-8.863204,-5.150114,4.360043,-8.404597,10.015437
